In [ ]:
## AKB Nov 2025: trying this out, changing directories

In [1]:
## Modules

import geopandas as gpd
import pandas as pd
import numpy as np

import json
import urllib.request
import geopandas as gpd
import pandas as pd
import numpy as np
import geopy.distance

In [2]:
## Utils
over_18_pop_by_zip = pd.read_csv(r"./Full US Data/over_18_pop_data.csv", index_col = 0)
cars_by_zip = pd.read_csv(r"./Full US Data/full_us_car_data.csv", index_col = 0)

def get_pop(zipcode):
    foo = 'ZCTA5 '+zipcode
    return int(over_18_pop_by_zip.loc[foo])

def get_cars(zipcode):
    foo = 'ZCTA5 '+zipcode
    return int(cars_by_zip.loc[foo])    

def get_zip(site, polls):
    return polls.loc[site]['address.zip']

def get_car_prop(site, polls):
    zipcode = get_zip(site, polls)
    pop = get_pop(zipcode)
    cars = get_cars(zipcode)
    if pop == 0:
        return 0
    else:
        return min(cars/pop, 1)


def calc_dtilde_matrix(polls, t_car, t_pub, t_walk, neighbours):
    N = polls.shape[0]
    dtilde_matrix = np.zeros((N,N))
    for i, a in polls.iterrows():
        for j, b in polls.iterrows():
            if i != j:
                walk = t_walk[i,j] + t_walk[j,i]
                pub = t_pub[i,j] + t_pub[j,i]
                car = t_car[i,j] + t_car[j,i]
                driver_min = min(walk, pub, car)
                non_driver_min = min(walk, pub)
                car_prop = get_car_prop(i, polls)
                dtilde_matrix[i,j] = car_prop*driver_min + (1-car_prop)*non_driver_min
    return dtilde_matrix

def calc_d_matrix(dtilde_matrix, polls, neighbours):
    N = polls.shape[0]
    d_matrix = np.zeros((N,N))
    for i, a in polls.iterrows():
        for j, b in polls.iterrows():
            if i != j:
                zipi = get_zip(i, polls)
                zipj = get_zip(j, polls)
                popi = get_pop(zipi)
                popj = get_pop(zipj)
                if popi == 0 and popj == 0:
                    d_matrix[i,j] = 0
                elif popi == 0:
                    d_matrix[i,j] = dtilde_matrix[j,i]
                elif popj == 0:
                    d_matrix[i,j] = dtilde_matrix[i,j]
                else:
                    d_matrix[i,j] = (1/(popi+popj))*(popi*dtilde_matrix[i,j] + popj*dtilde_matrix[j,i])
    return d_matrix

In [3]:
#walk stuff

def get_dist(a, b):
    origin = str(a.y)+','+str(a.x)
    destination = str(b.y)+','+str(b.x)
    dist = geopy.distance.geodesic(origin, destination)
    return dist.meters

def get_walk_time(a,b):
    dist = get_dist(a,b)
    return dist/1.38582

def calc_twalk_matrix(polls, neighbours):
    """
    Input:
    polls is a geodataframe storing polling locations as Points
    neighbours is a 0/1 array representing which sites we would like to compute distances between
    
    Returns: A matrix of walking times between polling sites
    """
    N = polls.shape[0]
    t_walk = np.zeros((N,N))
    for i, a in polls.iterrows():
        for j, b in polls.iterrows():
            if i != j:
                t_walk[i,j] = get_walk_time(a.geometry,b.geometry)
    return t_walk

In [4]:
import sysconfig
print(sysconfig.get_platform())

macosx-12.1-arm64


Example: SLC

In [5]:
slc_sites = gpd.read_file('Salt Lake City/slc_polls.geojson')
slc_sites.at[10,'address.zip'] = '84115'
slc_neighbours = np.ones((20,20))

In [8]:
slc_sites.columns.to_list()

['query_id',
 'placekey',
 'error',
 'pollingHours',
 'latitude',
 'longitude',
 'sources',
 'address.locationName',
 'address.line1',
 'address.city',
 'address.state',
 'address.zip',
 'notes',
 'startDate',
 'endDate',
 'address.line2',
 'address.line3',
 'id',
 'geometry']

In [68]:
get_dist(slc_sites.loc[0].geometry,slc_sites.loc[2].geometry)
#get_walk_time(slc_sites.loc[0]['geometry],slc_sites.loc[1][geometry])

9577.670574121512

In [51]:
slc_sites.head()

,query_id,placekey,error,pollingHours,latitude,longitude,sources,address.locationName,address.line1,address.city,address.state,address.zip,notes,startDate,endDate,address.line2,address.line3,id,geometry
0,40637,222@5qc-w55-xt9,None,7 am - 8 pm,40.711216,-112.110699,"[ { ""name"": ""Voting Information Project"", ""off...",Magna Senior Center,9228 W 2700 S,Magna,UT,84044,None,None,None,None,None,None,POINT (-112.1107 40.71122)
1,40642,@5qc-w8z-yy9,Invalid address,7 am - 8 pm,40.697153,-112.044563,"[ { ""name"": ""Voting Information Project"", ""off...",Hunter LDS Church,3450 S 6400 W,West Valley City,UT,84128,None,None,None,None,None,None,POINT (-112.04456 40.69715)
2,40644,224-222@5qc-w82-wrk,None,7 am - 8 pm,40.682686,-112.003759,"[ { ""name"": ""Voting Information Project"", ""off...",Hunter Library,4740 W 4100 S,West Valley City,UT,84120,None,None,None,None,None,None,POINT (-112.00376 40.68269)
3,40653,zzw-222@5qc-w89-54v,None,7 am - 8 pm,40.695408,-111.996823,"[ { ""name"": ""Voting Information Project"", ""off...",Sonrise Baptist Church,3550 S 4400 W,West Valley City,UT,84120,None,None,None,None,None,None,POINT (-111.99682 40.69541)
4,40661,222-222@5qc-w87-ysq,None,7 am - 8 pm,40.704098,-111.941413,"[ { ""name"": ""Voting Information Project"", ""off...",West Valley City Hall,3600 S Constitution Blvd,West Valley City,UT,84119,None,None,None,None,None,None,POINT (-111.94141 40.7041)


In [15]:
#slc_tcar = np.load("./Salt Lake City/slc_tcar_matrix_completed.npy")
#slc_tpub = np.load("./Salt Lake City/slc_tpub_matrix_completed.npy")

# v1: uses geopy.distance.geodesic
#
# v2: uses a saved matrix
slc_twalk_v1 = calc_twalk_matrix(slc_sites, slc_neighbours)
slc_twalk_v2 = np.load("./Salt Lake City/slc_walk.npy")

In [24]:
# Subtract off diagonal
slc_twalk_v2 = slc_twalk_v2-np.diag(np.diag(slc_twalk_v2))

In [29]:
print(slc_twalk_v1[0:4,0:4])
print(slc_twalk_v2[0:4,0:4])
print(slc_twalk_v2[0:4,0:4]/slc_twalk_v1[0:4,0:4])

[[   0.         4187.60708325 6911.19378716 7059.08161245]
 [4187.60708325    0.         2745.54928809 2914.98766986]
 [6911.19378716 2745.54928809    0.         1103.74709242]
 [7059.08161245 2914.98766986 1103.74709242    0.        ]]
[[   0.         4948.63450704 8030.87394366 7839.29647887]
 [4948.63450704    0.         3514.22535211 3084.44014085]
 [8030.87394366 3514.22535211    0.         1624.32183099]
 [7839.29647887 3084.44014085 1624.32183099    0.        ]]
[[       nan 1.18173324 1.16200966 1.1105264 ]
 [1.18173324        nan 1.27997169 1.05813145]
 [1.16200966 1.27997169        nan 1.47164313]
 [1.1105264  1.05813145 1.47164313        nan]]


/var/folders/cy/h13gdsk90z9b0q3yb1tc6v9c0000gn/T/ipykernel_885/1032366869.py:3: RuntimeWarning: invalid value encountered in divide
  print(slc_twalk_v2[0:4,0:4]/slc_twalk_v1[0:4,0:4])


In [38]:
# Set the unavailable ones to be Inf so they will never be used
slc_tcar = np.ones(slc_twalk_v1.shape)*np.inf
slc_tpub = np.ones(slc_twalk_v1.shape)*np.inf


In [40]:
slc_dtilde = calc_dtilde_matrix(slc_sites, slc_tcar, slc_tpub, slc_twalk_v1, slc_neighbours)
slc_d_matrix_v1 = calc_d_matrix(slc_dtilde, slc_sites, slc_neighbours)

slc_dtilde = calc_dtilde_matrix(slc_sites, slc_tcar, slc_tpub, slc_twalk_v2, slc_neighbours)
slc_d_matrix_v2 = calc_d_matrix(slc_dtilde, slc_sites, slc_neighbours)

/var/folders/cy/h13gdsk90z9b0q3yb1tc6v9c0000gn/T/ipykernel_885/973536524.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(over_18_pop_by_zip.loc[foo])
/var/folders/cy/h13gdsk90z9b0q3yb1tc6v9c0000gn/T/ipykernel_885/973536524.py:11: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(cars_by_zip.loc[foo])


In [47]:
print(slc_twalk_v1[0:4,0:4])
print(slc_d_matrix_v1[0:4,0:4])
# Ok, this clearly works out: distance is DOUBLE the walking time
print(slc_d_matrix_v1[0:4,0:4]/slc_twalk_v1[0:4,0:4])

[[   0.         4187.60708325 6911.19378716 7059.08161245]
 [4187.60708325    0.         2745.54928809 2914.98766986]
 [6911.19378716 2745.54928809    0.         1103.74709242]
 [7059.08161245 2914.98766986 1103.74709242    0.        ]]
[[    0.          8375.21416649 13822.38757432 14118.16322491]
 [ 8375.21416649     0.          5491.09857617  5829.97533972]
 [13822.38757432  5491.09857617     0.          2207.49418484]
 [14118.16322491  5829.97533972  2207.49418484     0.        ]]
[[nan  2.  2.  2.]
 [ 2. nan  2.  2.]
 [ 2.  2. nan  2.]
 [ 2.  2.  2. nan]]


/var/folders/cy/h13gdsk90z9b0q3yb1tc6v9c0000gn/T/ipykernel_885/3695192275.py:3: RuntimeWarning: invalid value encountered in divide
  print(slc_d_matrix_v1[0:4,0:4]/slc_twalk_v1[0:4,0:4])


In [50]:
np.save("./Salt Lake City/slc_d_matrix_updated_v1.npy", slc_d_matrix_v1) 
np.save("./Salt Lake City/slc_d_matrix_updated_v2.npy", slc_d_matrix_v2) 

Atlanta

In [8]:
atl_sites = gpd.read_file('Atlanta/atl_polls.geojson')
atl_neighbours = np.load("Atlanta/atl_neighbors.npy")

In [10]:
atl_tcar = np.load("Atlanta/atl_tcar_matrix_completed.npy")
atl_tpub = np.load("Atlanta/atl_tpub_matrix_completed.npy")
# atl_twalk = calc_twalk_matrix(atl_sites, atl_neighbours)
atl_twalk = np.load("walk_matrix_complete/atl_walk.npy")

In [11]:
atl_sites['address.zip'] = atl_sites['address.zip'].astype(float)
atl_sites['address.zip'] = atl_sites['address.zip'].astype(int)
atl_sites['address.zip'] = atl_sites['address.zip'].astype(str)

In [12]:
atl_sites.at[16, 'address.zip'] = '30312'

In [13]:
atl_dtilde = calc_dtilde_matrix(atl_sites, atl_tcar, atl_tpub, atl_twalk, atl_neighbours)
atl_d_matrix = calc_d_matrix(atl_dtilde, atl_sites, atl_neighbours)
np.save("atl_d_matrix_updated.npy", atl_d_matrix) 

LAC

In [14]:
lac_sites = gpd.read_file('Los Angeles/fixed_files/lac_NoCatalina_polls.geojson')
lac_neighbours = np.load("Los Angeles/lac_neighbors.npy")
for i,a in lac_sites.iterrows():
    if a['address.zip'] == '90095':
        lac_sites.at[i, 'address.zip'] = '90024'
    elif a['address.zip'] == '90506':
        lac_sites.at[i, 'address.zip'] = '90249'
    elif a['address.zip'] == '91371':
        lac_sites.at[i, 'address.zip'] = '91367'

In [15]:
lac_tcar = np.load("Los Angeles/fixed_files/lac_tcar_matrix_completed.npy")
lac_tpub = np.load("Los Angeles/fixed_files/lac_tpub_matrix_completed.npy")
# lac_twalk = calc_twalk_matrix(lac_sites, lac_neighbours)
lac_twalk = np.load("walk_matrix_complete\lac_walk.npy")

In [16]:
lac_dtilde = calc_dtilde_matrix(lac_sites, lac_tcar, lac_tpub, lac_twalk, lac_neighbours)
lac_d_matrix = calc_d_matrix(lac_dtilde, lac_sites, lac_neighbours)
np.save("lac_d_matrix_updated.npy", lac_d_matrix) 

Jacksonville

In [17]:
jax_sites = gpd.read_file('Jacksonville/jax_polls.geojson')
jax_neighbours = np.load("Jacksonville/jax_neighbors.npy")

In [18]:
jax_tcar = np.load("Jacksonville/jax_tcar_matrix_completed.npy")
jax_tpub = np.load("Jacksonville/jax_tpub_matrix_completed.npy")
# jax_twalk = calc_twalk_matrix(jax_sites, jax_neighbours)
jax_twalk = np.load("walk_matrix_complete\jax_walk.npy")

In [19]:
jax_dtilde = calc_dtilde_matrix(jax_sites, jax_tcar, jax_tpub, jax_twalk, jax_neighbours)
jax_d_matrix = calc_d_matrix(jax_dtilde, jax_sites, jax_neighbours)
np.save("jax_d_matrix_updated.npy", jax_d_matrix) 

Chicago

In [24]:
chc_sites = gpd.read_file('Chicago/chc_polls.geojson')
chc_neighbours = np.load("Chicago/chc_neighbors.npy")
chc_sites.at[397, 'address.zip'] = '60656'
chc_sites.at[453, 'address.zip'] = '60827'

chc_tcar = np.load("Chicago/chc_tcar_matrix_completed.npy")
chc_tpub = np.load("Chicago/chc_tpub_matrix_completed.npy")
# chc_twalk = calc_twalk_matrix(chc_sites, chc_neighbours)
chc_twalk = np.load("walk_matrix_complete\chc_walk.npy")

In [25]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(chc_sites['address.zip'])

0      60005
1      60007
2      60007
3      60007
4      60007
5      60007
6      60018
7      60714
8      60714
9      60018
10     60018
11     60018
12     60018
13     60018
14     60018
15     60068
16     60068
17     60068
18     60068
19     60068
20     60714
21     60106
22     60106
23     60106
24     60126
25     60131
26     60131
27     60131
28     60131
29     60176
30     60176
31     60176
32     60176
33     60176
34     60419
35     60601
36     60612
37     60601
38     60602
39     60606
40     60605
41     60605
42     60605
43     60605
44     60605
45     60606
46     60654
47     60607
48     60607
49     60607
50     60612
51     60607
52     60607
53     60608
54     60608
55     60608
56     60608
57     60608
58     60608
59     60608
60     60608
61     60608
62     60632
63     60609
64     60636
65     60632
66     60609
67     60609
68     60609
69     60609
70     60609
71     60610
72     60610
73     60610
74     60610
75     60614
76     60610

In [26]:
chc_sites.at[397,'address.line1']

'5220 North Canfield Avenue'

In [27]:
chc_dtilde = calc_dtilde_matrix(chc_sites, chc_tcar, chc_tpub, chc_twalk, chc_neighbours)
chc_d_matrix = calc_d_matrix(chc_dtilde, chc_sites, chc_neighbours)
np.save("chc_d_matrix_updated.npy", chc_d_matrix) 

NYC

In [28]:
manbronx_sites = gpd.read_file('NYC/manbronx_polls.geojson')
manbronx_neighbours = np.load("NYC/manbronx_neighbors.npy")
for i,a in manbronx_sites.iterrows():
    if a['address.zip'] == '10103':
        manbronx_sites.at[i, 'address.zip'] = '10019'

manbronx_tcar = np.load("NYC/manbronx_tcar_matrix_completed.npy")
manbronx_tpub = np.load("NYC/manbronx_tpub_matrix_completed.npy")
# manbronx_twalk = calc_twalk_matrix(manbronx_sites, manbronx_neighbours)
manbronx_twalk = np.load("walk_matrix_complete\manbronx_walk.npy")

manbronx_dtilde = calc_dtilde_matrix(manbronx_sites, manbronx_tcar, manbronx_tpub, manbronx_twalk, manbronx_neighbours)
manbronx_d_matrix = calc_d_matrix(manbronx_dtilde, manbronx_sites, manbronx_neighbours)
np.save("manbronx_d_matrix_updated.npy", manbronx_d_matrix) 

In [32]:
queensbrook_sites = gpd.read_file('NYC/queensbrook_polls.geojson')
queensbrook_neighbours = np.load("NYC/queensbrook_neighbors.npy")

queensbrook_tcar = np.load("NYC/queensbrook_tcar_matrix_completed.npy")
queensbrook_tpub = np.load("NYC/queensbrook_tpub_matrix_completed.npy")
# queensbrook_twalk = calc_twalk_matrix(queensbrook_sites, queensbrook_neighbours)
queensbrook_twalk = np.load("walk_matrix_complete\queensbrook_walk.npy")

In [33]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(queensbrook_sites['address.zip'])

0      11249
1      11233
2      11001
3      11001
4      11001
5      11001
6      11004
7      11005
8      11040
9      11040
10     11040
11     11040
12     11040
13     11101
14     11101
15     11104
16     11106
17     11101
18     11101
19     11101
20     11105
21     11102
22     11102
23     11102
24     11105
25     11103
26     11377
27     11104
28     11104
29     11377
30     11105
31     11106
32     11106
33     11101
34     11101
35     11109
36     11201
37     11201
38     11201
39     11201
40     11201
41     11201
42     11203
43     11203
44     11203
45     11203
46     11203
47     11203
48     11204
49     11204
50     11214
51     11204
52     11204
53     11204
54     11205
55     11205
56     11205
57     11205
58     11205
59     11206
60     11206
61     11206
62     11206
63     11206
64     11206
65     11206
66     11208
67     11207
68     11207
69     11207
70     11207
71     11212
72     11208
73     11208
74     11208
75     11208
76     11207

In [34]:
queensbrook_dtilde = calc_dtilde_matrix(queensbrook_sites, queensbrook_tcar, queensbrook_tpub, queensbrook_twalk, queensbrook_neighbours)
queensbrook_d_matrix = calc_d_matrix(queensbrook_dtilde, queensbrook_sites, queensbrook_neighbours)
np.save("queensbrook_d_matrix_updated.npy", queensbrook_d_matrix) 

In [35]:
stat_sites = gpd.read_file('NYC/stat_polls.geojson')
stat_neighbours = np.load("NYC/stat_neighbors.npy")

stat_tcar = np.load("NYC/stat_tcar_matrix_completed.npy")
stat_tpub = np.load("NYC/stat_tpub_matrix_completed.npy")
# stat_twalk = calc_twalk_matrix(stat_sites, stat_neighbours)
stat_twalk = np.load("walk_matrix_complete\stat_walk.npy")

stat_dtilde = calc_dtilde_matrix(stat_sites, stat_tcar, stat_tpub, stat_twalk, stat_neighbours)
stat_d_matrix = calc_d_matrix(stat_dtilde, stat_sites, stat_neighbours)
np.save("stat_d_matrix_updated.npy", stat_d_matrix) 

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(over_18_pop_by_zip)

In [ ]:
np.save("slc_twalk_matrix_completed.npy", slc_twalk)
np.save("atl_twalk_matrix_completed.npy", atl_twalk)
np.save("lac_twalk_matrix_completed.npy", lac_twalk)
np.save("chc_twalk_matrix_completed.npy", chc_twalk)
np.save("jax_twalk_matrix_completed.npy", jax_twalk)
np.save("queensbrook_twalk_matrix_completed.npy", queensbrook_twalk)
np.save("stat_twalk_matrix_completed.npy", stat_twalk)
np.save("manbronx_twalk_matrix_completed.npy", manbronx_twalk)